# Depth Detector

### 1) Place two tennis balls directly underneath one another. One on the ground and one on top of the object of interest.

### 2) Take an image as close to 90 degrees as possible

### 3) Upload the image and find the depth

In [1]:
import numpy as np
# import argparse
import cv2
from scipy.spatial import distance as dist
from imutils import contours
import imutils

from ipywidgets import *
from PIL import Image

In [2]:
# # construct the argument parse and parse the arguments
# ap = argparse.ArgumentParser()
# ap.add_argument("-i", "--image", required=True,
#     help="path to the input image")
# ap.add_argument("-w", "--width", type=float, required=True,
#     help="width of the left-most object in the image (in centimeters)")
# args = vars(ap.parse_args())

In [3]:
def midpoint(ptA, ptB):
    return ((ptA[0] + ptB[0]) * 0.5, (ptA[1] + ptB[1]) * 0.5)

def rec_center_point(x,y,w,h):
    xm = x + (w/2)
    ym = y + (h/2)
    return xm, ym

def distance(x1,y1,x2,y2):
    dx = abs(x1-x2)
    dy = abs(y2-y1)
    dist = np.sqrt(dx**2 + dy**2)
    return dist

def normalize_distance(w1,h1,w2,h2,width):
    avg_dim = np.mean([w1,h1,w2,h2])
    multiplier = width/avg_dim
    return multiplier

In [4]:
def color_extractor(image_data):
    nparr = np.frombuffer(image_data, np.uint8)
    image = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
    
    out_img_og.clear_output()
    with out_img_og:
        disp_im = Image.fromarray(np.flip(image, axis=-1))
        disp_im.thumbnail([450,450])
        display(disp_im)
           
    ## convert to hsv, hsv is easier than bgr range
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, (36, 25, 25), (70, 255,255))
    
    ## slice the selected color range
    imask = mask>0
    color_sliced_image = np.zeros_like(image, np.uint8)
    color_sliced_image[imask] = image[imask]
       
    return image, color_sliced_image

In [5]:
# Create widgets
btn1_upload = FileUpload(description='Select Image')
out_img_og = Output()
btn_load = Button(description='Find Depth')
out_img_measure = Output()

item_width = 6.9 # width in cm

def detect_depth(change):
    # Extract color of interest from selected image
    og_img, extracted_img = color_extractor(btn1_upload.data[-1])
    
    # Make a copy of the extracted image
    image = extracted_img.copy() 

    # Grayscale and denoise
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    edged = cv2.Canny(gray, 50, 200)
    edged = cv2.dilate(edged, None, iterations=1)
    edged = cv2.erode(edged, None, iterations=1)

    # Finding Contours
    # Use a copy of the image e.g. edged.copy()
    # since findContours alters the image
    contours, hierarchy = cv2.findContours(edged, 
        cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    # plt.figure()
    # plt.imshow(edged)
    # plt.show()

    # Draw all contours
    # -1 signifies drawing all contours
    cv2.drawContours(image, contours, -1, (0, 255, 0), 3)
    
    # Sort contours from largest to smallest
    contours = sorted(contours, key=cv2.contourArea, reverse=True)
    c1 = contours[0] # largest contour
    c2 = contours[1] # second largest contour

    # Draw bounding rectangles on 2 largest contours
    x1,y1,w1,h1 = cv2.boundingRect(c1)
    cv2.rectangle(image,(x1,y1),(x1+w1,y1+h1),(255,0,0),20) # red
    cv2.rectangle(og_img,(x1,y1),(x1+w1,y1+h1),(255,0,0),20) # red

    x2,y2,w2,h2 = cv2.boundingRect(c2)
    cv2.rectangle(image,(x2,y2),(x2+w2,y2+h2),(0,0,255),20) # blue
    cv2.rectangle(og_img,(x2,y2),(x2+w2,y2+h2),(0,0,255),20) # blue

    # # show the images
    # plt.figure()
    # plt.imshow(image)
    # plt.show()
    
    # define center points of rectangles
    xm1,ym1 = rec_center_point(x1,y1,w1,h1)
    xm2,ym2 = rec_center_point(x2,y2,w2,h2)
    
    # find the unnormalized distance between center points
    D = dist.euclidean([xm1,ym1],[xm2,ym2])
    
    # find multiplier with which to normalize the distance
    multiplier = normalize_distance(w1,h1,w2,h2,item_width)
    
    # define midpoint between two bounding rectangles, this is where we write the depth
    (mX, mY) = midpoint((xm1, ym1), (xm2, ym2))

    # draw connecting line and depth on original image
    cv2.circle(og_img, (int(xm1), int(ym1)), 5, (0,255,0), -1)
    cv2.circle(og_img, (int(xm2), int(ym2)), 5, (0,255,0), -1)
    cv2.line(og_img, (int(xm1), int(ym1)), (int(xm2), int(ym2)), (0,255,0), 2)
    cv2.putText(og_img, "{:.1f}cm".format(D*multiplier), (int(mX), int(mY - 10)),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (0,255,0), 4)
    
    # print distance to screen
    print('Distance: ' + str(round(D*multiplier,1)) + ' cm')
    
    # display image in jupyter widget
    out_img_measure.clear_output()
    with out_img_measure:
        disp_im = Image.fromarray(np.flip(og_img, axis=-1))
        disp_im.thumbnail([450,450])
        display(disp_im)
    
btn_load.on_click(detect_depth)

In [6]:
VBox([HBox([btn1_upload, btn_load]), 
      HBox([out_img_og, out_img_measure])])